In [2]:
pip install yfinance


  Using cached yfinance-0.2.55-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.9-cp310-cp310-macosx_11_0_arm64.whl
Using cached yfinance-0.2.55-py2.py3-none-any.whl (109 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.3/187.3 kB 2.5 MB/s eta 0:00:00 0:00:01
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import yfinance as yf
import pandas as pd
import os

# Create folder for raw data
os.makedirs("data/raw", exist_ok=True)

# Futures symbols
futures = {
    "Crude Oil": "CL=F",
    "Gold": "GC=F",
    "S&P 500 E-mini": "ES=F"
}

# Fetch and save data
for name, symbol in futures.items():
    print(f"Fetching data for {name} ({symbol})...")
    data = yf.download(symbol, start="2015-01-01", end="2025-04-22", interval="1d")
    
    # Fix: reset index to make date a column
    data.reset_index(inplace=True)

    # Save to CSV
    file_path = f"data/raw/{symbol.replace('=','_')}.csv"
    data.to_csv(file_path, index=False)
    print(f"✅ Saved to {file_path}\n")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching data for Crude Oil (CL=F)...
✅ Saved to data/raw/CL_F.csv

Fetching data for Gold (GC=F)...
✅ Saved to data/raw/GC_F.csv

Fetching data for S&P 500 E-mini (ES=F)...
✅ Saved to data/raw/ES_F.csv



Data Cleaning & SQL Insertion
🧹 Step 1: Clean & Standardize Data
Let’s write a Python script to:

Load each CSV

Clean missing data

Standardize column names

Add symbol column

Save a cleaned version and insert into a SQL table




In [4]:
pip install sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Create output folders if they don't exist
os.makedirs("data/processed", exist_ok=True)
os.makedirs("data/raw", exist_ok=True)

# Set up SQLite database
engine = create_engine('sqlite:///data/futures_data.db')

# Define futures files and symbols
files = {
    'CL=F': 'data/raw/CL_F.csv',
    'GC=F': 'data/raw/GC_F.csv',
    'ES=F': 'data/raw/ES_F.csv'
}

# Define expected numeric columns
numeric_columns = ['open', 'high', 'low', 'close', 'volume']

# Process each file
for symbol, file_path in files.items():
    print(f"📦 Processing {symbol} from {file_path}...")

    try:
        # Load the raw CSV
        df = pd.read_csv(file_path)

        # Print original column names for inspection
        print(f"🔍 Original columns: {df.columns.tolist()}")

        # Normalize column names
        df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

        # Check for 'date' column
        if 'date' not in df.columns:
            raise ValueError(f"❌ 'date' column not found in: {file_path}")

        # Convert date column to datetime
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

        # Convert numeric columns explicitly
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Drop rows with any missing or malformed data
        df.dropna(subset=['date'] + numeric_columns, inplace=True)

        # Add symbol column
        df['symbol'] = symbol

        # Reorder columns for consistency
        ordered_cols = ['date'] + numeric_columns + ['symbol']
        df = df[ordered_cols]

        # Save cleaned data
        cleaned_path = file_path.replace('raw', 'processed')
        df.to_csv(cleaned_path, index=False)
        print(f"✅ Cleaned data saved to: {cleaned_path}")

        # Insert into SQL database
        df.to_sql("futures_data", con=engine, if_exists="append", index=False)
        print(f"📥 Inserted {len(df)} rows into 'futures_data' table.\n")

    except Exception as e:
        print(f"❗ Error processing {symbol}: {e}\n")


📦 Processing CL=F from data/raw/CL_F.csv...
🔍 Original columns: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
✅ Cleaned data saved to: data/processed/CL_F.csv
📥 Inserted 2589 rows into 'futures_data' table.

📦 Processing GC=F from data/raw/GC_F.csv...
🔍 Original columns: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
✅ Cleaned data saved to: data/processed/GC_F.csv
📥 Inserted 2588 rows into 'futures_data' table.

📦 Processing ES=F from data/raw/ES_F.csv...
🔍 Original columns: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
✅ Cleaned data saved to: data/processed/ES_F.csv
📥 Inserted 2590 rows into 'futures_data' table.



data is now:

✅ Cleaned

✅ Stored in SQLite (futures_data.db)

✅ Structured for analysis with symbol, date, open, high, low, close, volume

